### Setup

In [35]:
import sys
import os

# Add the project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [36]:
from src.idspy.core.pipeline import Pipeline, PipelineEvent, hook, FitAwarePipeline
from src.idspy.core.state import State
from src.idspy.core.step import Step, FitAwareStep, Repeat

### Example `Steps`

In [ ]:
class Load(Step):
    def __init__(self): super().__init__(provides=["data"])

    def run(self, state: State) -> None:
        state["data"] = [1, 2, 3]


class Sum(Step):
    def __init__(self): super().__init__(requires=["data"], provides=["sum"])

    def run(self, state: State) -> None:
        state["sum"] = sum(state["data"])


class MeanCenter(FitAwareStep):
    def __init__(self): super().__init__(requires=["data"], provides=["data"])

    def fit_impl(self, state: State) -> None:
        xs = state["data"];
        state["preproc.mean"] = sum(xs) / len(xs)

    def run(self, state: State) -> None:
        m = state["preproc.mean"]
        state["data"] = [x - m for x in state["data"]]


class Accumulate(Step):
    def __init__(self): super().__init__(requires=["sum"], provides=["tot"])

    def run(self, state: State) -> None:
        state["tot"] = state.get("tot", 0) + state["sum"]

#NUOVE AGGIUNTE
class Multiply(Step):
    def __init__(self, factor: int):
        super().__init__(requires=["data"], provides=["mul"])
        self.factor = factor

    def run(self, state: State) -> None:
        state["mul"] = [x * self.factor for x in state["data"]]

class StandardScaler(FitAwareStep):
    def __init__(self):
        super().__init__(requires=["data"], provides=["data"])
        self.std = 1.0

    def fit_impl(self, state: State):
        data = state["data"]
        mean = sum(data) / len(data)
        var = sum((x - mean) ** 2 for x in data) / len(data)
        self.std = var**0.5 if var > 0 else 1.0
        state["preproc.std"] = self.std

    def run(self, state: State):
        state["data"] = [x / self.std for x in state["data"]]


class ConditionalSum(Step):
    def __init__(self):
        super().__init__(requires=["data"], provides=["c_sum"])

    def run(self, state: State):
        data = state["data"]
        state["c_sum"] = sum(data) if len(data) > 2 else None

class Pitagora(Step):
    def __init__(self):
        super().__init__(requires=["data"], provides=["pitagora"])

    def run(self, state: State):
        
        if len(state["data"]) < 2:
            state["pitagora"] = None
        else:
            list_pitagora = []
            for i in range(len(state["data"]) - 1):
                a = state["data"][i]
                b = state["data"][i + 1]
                list_pitagora.append(int((a**2 + b**2) ** 0.5))
            state["pitagora"] = list_pitagora     
#FINE NUOVE AGGIUNTE


### Build a `Pipeline` with Custom `@decorator` Hooks

In [58]:
class MyPipeline(Pipeline):
    @hook(PipelineEvent.PIPELINE_START)
    def _start(self, state: State) -> None:
        print("[pipeline] start")

    @hook(PipelineEvent.BEFORE_STEP, priority=-1)
    def _before(self, step: Step, state: State, *, index: int) -> None:
        print(f"[pipeline] before {index}: {step.name}")

    @hook(PipelineEvent.AFTER_STEP)
    def _after(self, step: Step, state: State, *, index: int) -> None:
        print(f"[pipeline] after {index}:  {step.name}")
        
    @hook(PipelineEvent.PIPELINE_END)
    def _end(self, state: State) -> None:
        print("[pipeline] end")


s = State()
p = MyPipeline([Load(), Sum()], name="Plain")
p(s)
print(s.to_dict())
# [pipeline] start
# [pipeline] before 0: Load
# [pipeline] after 0:  Load
# [pipeline] before 1: Sum
# [pipeline] after 1:  Sum
# [pipeline] end
# {'data': [1, 2, 3], 'sum': 6}

#NUOVE AGGIUNTE
class MyPipeline2(Pipeline):
    @hook(PipelineEvent.PIPELINE_START)
    def _start(self, state: State) -> None:
        print("[pipeline_1] start")

    @hook(PipelineEvent.BEFORE_STEP, priority=-1)
    def _before(self, step: Step, state: State, *, index: int) -> None:
        print(f"[pipeline_1] before {index}: {step.name}")

    @hook(PipelineEvent.AFTER_STEP)
    def _after(self, step: Step, state: State, *, index: int) -> None:
        print(f"[pipeline_1] after {index}:  {step.name}")
        print(f"    stato parziale: {state.to_dict()}")
        
    @hook(PipelineEvent.PIPELINE_END)
    def _end(self, state: State) -> None:
        print("[pipeline_1] end")
        
s1 = State()
p1 = MyPipeline2([Load(), Sum(), Accumulate(), Multiply(factor=10), Pitagora()], name="MP2")
p1(s1)
print(s1.to_dict())

[pipeline] start
[pipeline] before 0: Load
[pipeline] after 0:  Load
[pipeline] before 1: Sum
[pipeline] after 1:  Sum
[pipeline] end
{'data': [1, 2, 3], 'sum': 6}
[pipeline_1] start
[pipeline_1] before 0: Load
[pipeline_1] after 0:  Load
    stato parziale: {'data': [1, 2, 3]}
[pipeline_1] before 1: Sum
[pipeline_1] after 1:  Sum
    stato parziale: {'data': [1, 2, 3], 'sum': 6}
[pipeline_1] before 2: Accumulate
[pipeline_1] after 2:  Accumulate
    stato parziale: {'data': [1, 2, 3], 'sum': 6, 'tot': 6}
[pipeline_1] before 3: Multiply
[pipeline_1] after 3:  Multiply
    stato parziale: {'data': [1, 2, 3], 'sum': 6, 'tot': 6, 'mul': [10, 20, 30]}
[pipeline_1] before 4: Pitagora
[pipeline_1] after 4:  Pitagora
    stato parziale: {'data': [1, 2, 3], 'sum': 6, 'tot': 6, 'mul': [10, 20, 30], 'pitagora': [2, 3]}
[pipeline_1] end
{'data': [1, 2, 3], 'sum': 6, 'tot': 6, 'mul': [10, 20, 30], 'pitagora': [2, 3]}


### Build a `FitAwarePipeline` that fits all `FitAwareSteps` prior to execution

In [ ]:
s = State({"data": [1.0, 2.0, 3.0]})
fp = FitAwarePipeline([MeanCenter(), Sum()], name="FitPipe", refit=False)
fp(s)
print(s.to_dict())
# {'data': [-1.0, 0.0, 1.0], 'preproc.mean': 2.0, 'sum': 0.0}

# secondo run senza refit (usa ancora mean=2.0)
s["data"] = [2.0, 4.0, 6.0]
fp(s)
print(s.to_dict())
# {'data': [0.0, 2.0, 4.0], 'preproc.mean': 2.0, 'sum': 6.0}

# pipeline con refit=True
s2 = State({"data": [2.0, 4.0, 6.0]})
fp_refit = FitAwarePipeline([MeanCenter(), Sum()], name="FitPipeRefit", refit=True)
fp_refit(s2)
print(s2.to_dict())
# {'data': [-2.0, 0.0, 2.0], 'preproc.mean': 4.0, 'sum': 0.0}



#NUOVA AGGIUNTA
s1 = State({"data": [3.0, 4.0, 5.0]})
fp = FitAwarePipeline([MeanCenter(), StandardScaler(), Sum()], name="FittedPipe", refit=True)
fp(s1)
print(s1.to_dict())
# somma = 3+4+5 = 12; len = 3; ==> mean = 4.0;
# std = sqrt(((3-4)^2 + (4-4)^2 + (5-4)^2)/3) = sqrt((1+0+1)/3) = sqrt(2/3) = 0.816496580927726
# data normalizzato = [ (3-4)/std, (4-4)/std, (5-4)/std ] = [-1.224744871391589, 0.0, 1.224744871391589]
# preproc.std = 0.816496580927726 
# {'data': [-1.224744871391589, 0.0, 1.224744871391589], 'preproc.mean': 4.0, 'preproc.std':0.816  'sum': 0.0}


#Altra nuova aggiunta
s1["data"] = [10.0, 24.0]
fp = FitAwarePipeline([MeanCenter(), ConditionalSum(), Sum(), Multiply(factor=10)], name="FitPipeCond", refit=True)
fp(s1)
print(s1.to_dict())
# somma = 10+24 = 34; len = 2; ==> mean = 17.0;
# {'data': [-7.0, 7.0], 'preproc.mean': 17.0, 'c_sum':NONE, 'sum': 0.0, 'mul'= [-70.0, 70.0]}



### Repeat the Pipeline `count` times

In [ ]:
p = Pipeline([Load(), Sum(), Accumulate()])
rp = Repeat(p, count=3)
s = State()
rp(s)
print(s.to_dict())
# 1° iter: data=[1,2,3], sum=6, tot=6
# 2° iter: data=[1,2,3], sum=6, tot=12
# 3° iter: data=[1,2,3], sum=6, tot=18
# {'data': [1, 2, 3], 'sum': 6, 'tot': 18}